In [2]:
from keras.models import Model
from keras.preprocessing import image
from keras.optimizers import SGD
from keras.applications.vgg16 import VGG16
import matplotlib.pyplot as plt
import numpy as np

In [3]:
model = VGG16(weights='imagenet', include_top=True) # Will download the 560MB trained model.
sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(optimizer=sgd, loss='categorical_crossentropy')

553467904/553467096 [==============================] - 199s 0us/step
